In [1]:
import torch
from torch import nn
import numpy as np
from lgg_model import *
from gensim.models import Word2Vec

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
device

device(type='cpu')

In [3]:
word_model_path = input("请使用想使用的词汇库(无需添加后缀)：")
word_model_path = 'word_model_paths/' + word_model_path
word_model = Word2Vec.load(word_model_path)
wv = word_model.wv

In [4]:
lgg_model_path = input("请输入想使用的语言模型(无需添加后缀)：")
lgg_model_path = 'lgg_model_paths/' + lgg_model_path
lgg_model = torch.load(lgg_model_path)
lgg_model.eval()

vanilla_GRU(
  (Embedding): Embedding(2045, 50)
  (GRU): GRU(50, 100, num_layers=2, batch_first=True, dropout=0.5)
  (Linear): Linear(in_features=100, out_features=2045, bias=True)
)

In [12]:
"""caution: this block is for testing for cangtou poem"""

punc = ['，','。','？','！','；','【','】', '）', '（', ' ', '\n', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0']

del_lst = []
head = input("请输入“头”：")
head = head + '。'
lst = list(head) # default to be 4 Chinese characters

In [13]:
# test if all the characters are in the corpus
print(head[:-1])
for i in head[:-1]:
    print(i, wv.key_to_index[i])

北京大学清华大学
北 83
京 312
大 231
学 590
清 88
华 169
大 231
学 590


In [14]:
# words = input("请输入初始文本：")
# del_lst = []
# lst = list(words)

for i in lst:
    if i not in wv.key_to_index:
        del_lst.append(i)
for i in del_lst:
    lst.remove(i)

data = np.array([])
# for i in lst:
#     data = np.append(data, wv.key_to_index[i])

count = (len(head)-1) * 5
# count = int(input("请输入想要生成的字数："))
# while count % 5 != 0:
#     count = int(input("请重新输入："))

print("输入：", head[:-1])
print("输出：")

# for i in lst:
#     print(i, end='')
idx = wv.key_to_index[lst[0]]

i = 0

while i < count:
    i += 1
    new_word = wv.index_to_key[idx]
    print(new_word, end='')

    # lst.append(new_word)
    data = np.append(data, idx)
    data = np.stack((data,))

    x = torch.Tensor(data)
    x = x.to(torch.long)
    y = lgg_model(x)[0][-1]
    p = y.detach().numpy()
    p = softmax(p)

    idx = np.random.choice(np.arange(len(wv)), p=p)
    
    if (i % 5 != 0):
        while wv.index_to_key[idx] in punc:
            idx = np.random.choice(np.arange(len(wv)), p=p)

    if i % 5 == 0 and i != 0: # default for 5 characters in a sentence
        if i % 10 == 0:
            idx = wv.key_to_index['。']

        else:
            idx = wv.key_to_index['，']

        data = np.append(data, idx)
        data = np.stack((data,))

        new_word = wv.index_to_key[idx]
        print(new_word, end='')

        if i % 10 == 0:
            print()

        head_idx = i // 5
        head_word = head[head_idx]
        idx = wv.key_to_index[head_word]

    # new_word = wv.index_to_key[idx]
    # print(new_word, end='')

    # lst.append(new_word)
    # data = np.append(data, idx)

# print('\nGeneration finished.')



输入： 北京大学清华大学
输出：
北枝林粉红，京师春辉照。大条风备乐，学府九如伤。
清泠其千子，华浦荷晓寒。大运馀胜侣，学府玉壶妆。


In [1]:
%run generator.py

输入： 秋风萧瑟
输出：
秋登白月来，风劲浅残香。
萧条深微续，瑟事尚兽虚。
